## 1. Setup e Importación

In [ ]:
# Imports
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Custom modules
from recommendations import TrackRecommender, create_recommender_from_data
from playlist_generator import PlaylistGenerator, create_playlist_generator
from models import (
    recommendation_score, 
    evaluate_recommendation_quality,
    plot_recommendation_evaluation,
    analyze_playlist_diversity,
    plot_playlist_summary
)

# Config
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")
np.random.seed(42)

print("✓ Módulos importados correctamente")

## 2. Cargar Datos

In [ ]:
# Cargar datos
data_path = Path('../data/features/listening_history_with_features.parquet')

if not data_path.exists():
    print("⚠️ Usando datos sintéticos de demo")
    data_path = Path('../data/demo/synthetic_spotify_data.parquet')

df = pd.read_parquet(data_path)
print(f"✓ Datos cargados: {df.shape}")
print(f"\nColumnas disponibles:")
print(df.columns.tolist())

# Info básica
print(f"\n📊 Información del dataset:")
print(f"  Total reproducciones: {len(df):,}")
print(f"  Tracks únicos: {df['spotify_track_uri'].nunique():,}")
if 'master_metadata_album_artist_name' in df.columns:
    print(f"  Artistas únicos: {df['master_metadata_album_artist_name'].nunique():,}")
print(f"  Período: {df['ts'].min()} a {df['ts'].max()}")

## 3. Sistema de Recomendación

### 3.1 Inicializar Recomendador

In [ ]:
# Crear recomendador
recommender = create_recommender_from_data(df)

print("\n📈 Features de tracks construidos:")
print(recommender.track_features.head())

print("\n👤 Perfil de usuario:")
for key, value in recommender.user_profile.items():
    if isinstance(value, (list, dict)) and len(str(value)) > 100:
        print(f"  {key}: [datos extensos...]")
    else:
        print(f"  {key}: {value}")

### 3.2 Tracks Similares

Encontrar tracks similares a uno específico.

In [ ]:
# Elegir un track popular para el ejemplo
popular_track = df['spotify_track_uri'].value_counts().index[0]

print(f"🎵 Buscando tracks similares a: {popular_track}")

# Obtener tracks similares
similar_tracks = recommender.get_similar_tracks(popular_track, n=10)

print("\n✨ Top 10 tracks similares:")
display_cols = ['spotify_track_uri', 'track_name', 'artist', 'similarity_score', 
                'play_count', 'completion_rate']
display_cols = [c for c in display_cols if c in similar_tracks.columns]
print(similar_tracks[display_cols])

### 3.3 Recomendaciones Contextuales

Recomendar tracks basados en hora del día y día de la semana.

In [ ]:
# Recomendaciones para diferentes contextos
from datetime import datetime

# Hora actual
current_hour = datetime.now().hour
current_day = datetime.now().weekday()

print(f"🕐 Hora actual: {current_hour}h, Día: {current_day} (0=Lunes)")

# Recomendaciones para ahora
recs_now = recommender.recommend_for_context(
    hour=current_hour,
    day_of_week=current_day,
    n=15
)

print("\n✨ Recomendaciones para AHORA:")
display_cols = ['track_name', 'artist', 'final_score', 'popularity', 'completion_rate']
display_cols = [c for c in display_cols if c in recs_now.columns]
print(recs_now[display_cols].head(10))

# Comparar con recomendaciones para la mañana
recs_morning = recommender.recommend_for_context(hour=8, n=10)

print("\n☀️ Recomendaciones para la MAÑANA (8am):")
print(recs_morning[display_cols].head(10))

### 3.4 Recomendaciones Basadas en Favoritos

In [ ]:
# Recomendar basado en favoritos
recs_favorites = recommender.recommend_based_on_favorites(n=20)

print("💖 Recomendaciones basadas en tus favoritos:")
display_cols = ['track_name', 'artist', 'recommendation_score', 
                'similarity_score', 'popularity']
display_cols = [c for c in display_cols if c in recs_favorites.columns]
print(recs_favorites[display_cols].head(15))

### 3.5 Tracks Resistentes a Skips

In [ ]:
# Tracks con baja probabilidad de skip
recs_skip_resistant = recommender.recommend_skip_resistant(n=20)

print("🎯 Tracks que nunca querrás skipear:")
display_cols = ['track_name', 'artist', 'skip_resistant_score', 
                'completion_rate', 'skip_rate', 'play_count']
display_cols = [c for c in display_cols if c in recs_skip_resistant.columns]
print(recs_skip_resistant[display_cols].head(15))

### 3.6 Recomendaciones Híbridas

Combina múltiples estrategias para mejores resultados.

In [ ]:
# Recomendaciones híbridas
recs_hybrid = recommender.get_recommendations(
    strategy='hybrid',
    n=25,
    hour=current_hour,
    day_of_week=current_day
)

print("🌟 Recomendaciones HÍBRIDAS (mejor balance):")
display_cols = ['track_name', 'artist', 'hybrid_score', 'popularity', 'completion_rate']
display_cols = [c for c in display_cols if c in recs_hybrid.columns]
print(recs_hybrid[display_cols].head(20))

### 3.7 Evaluación de Recomendaciones

In [ ]:
# Evaluar calidad de recomendaciones
eval_metrics = recommender.evaluate_recommendations(recs_hybrid)

print("📊 Métricas de calidad:")
for metric, value in eval_metrics.items():
    if isinstance(value, float):
        print(f"  {metric}: {value:.2%}")
    else:
        print(f"  {metric}: {value}")

# Visualizar
if len(eval_metrics) > 0:
    fig = plot_recommendation_evaluation(eval_metrics)
    fig.show()

## 4. Generación Automática de Playlists

### 4.1 Inicializar Generador

In [ ]:
# Crear generador de playlists
generator = create_playlist_generator(df, generate_all=True)

### 4.2 Listar Playlists Generadas

In [ ]:
# Ver todas las playlists
playlists = generator.list_playlists()

print(f"\n🎵 {len(playlists)} playlists generadas:\n")
for name, count in playlists:
    print(f"  📀 {name}: {count} tracks")

### 4.3 Explorar Playlists Específicas

In [ ]:
# Morning Energy playlist
if 'Morning Energy' in generator.playlists:
    morning_playlist = generator.get_playlist('Morning Energy')
    
    print("☀️ MORNING ENERGY PLAYLIST:")
    display_cols = ['track_name', 'artist', 'typical_hour', 'play_count', 'completion_rate']
    display_cols = [c for c in display_cols if c in morning_playlist.columns]
    print(morning_playlist[display_cols].head(15))
    
    # Analizar diversidad
    diversity = analyze_playlist_diversity(morning_playlist)
    print("\n📊 Análisis de diversidad:")
    for key, value in diversity.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.2f}")
        else:
            print(f"  {key}: {value}")

In [ ]:
# Never Skip Hits playlist
if 'Never Skip Hits' in generator.playlists:
    never_skip = generator.get_playlist('Never Skip Hits')
    
    print("🎯 NEVER SKIP HITS PLAYLIST:")
    display_cols = ['track_name', 'artist', 'skip_rate', 'play_count', 'completion_rate']
    display_cols = [c for c in display_cols if c in never_skip.columns]
    print(never_skip[display_cols].head(15))
    
    # Visualizar
    if len(never_skip) > 0:
        fig = plot_playlist_summary(never_skip, title='Never Skip Hits - Resumen')
        fig.show()

In [ ]:
# All-Time Favorites
if 'All-Time Favorites' in generator.playlists:
    favorites = generator.get_playlist('All-Time Favorites')
    
    print("💎 ALL-TIME FAVORITES PLAYLIST:")
    display_cols = ['track_name', 'artist', 'play_count', 'typical_hour']
    display_cols = [c for c in display_cols if c in favorites.columns]
    print(favorites[display_cols].head(20))
    
    # Top artistas
    if 'artist' in favorites.columns:
        print("\n🎤 Top artistas en favoritos:")
        top_artists = favorites['artist'].value_counts().head(10)
        print(top_artists)

### 4.4 Comparar Playlists

In [ ]:
# Comparar playlists temporales
temporal_playlists = ['Morning Energy', 'Afternoon Vibes', 'Evening Chill', 'Late Night']

comparison_data = []
for name in temporal_playlists:
    if name in generator.playlists:
        playlist = generator.get_playlist(name)
        diversity = analyze_playlist_diversity(playlist)
        
        comparison_data.append({
            'Playlist': name,
            'N Tracks': diversity.get('n_tracks', 0),
            'N Artists': diversity.get('n_artists', 0),
            'Artist Diversity': diversity.get('artist_diversity', 0),
            'Avg Skip Rate': diversity.get('avg_skip_rate', 0)
        })

if comparison_data:
    comparison_df = pd.DataFrame(comparison_data)
    print("\n📊 Comparación de playlists temporales:")
    print(comparison_df)
    
    # Visualizar
    fig = px.bar(comparison_df, x='Playlist', y='Artist Diversity',
                 title='Diversidad de Artistas por Playlist',
                 color='Artist Diversity',
                 color_continuous_scale='Viridis')
    fig.update_layout(yaxis_tickformat='.0%')
    fig.show()

### 4.5 Análisis de Clusters

In [ ]:
# Analizar playlists de clustering
cluster_playlists = [name for name in generator.playlists.keys() 
                     if 'Cluster Mix' in name]

if cluster_playlists:
    print(f"\n🧩 {len(cluster_playlists)} playlists de clustering generadas:")
    
    for name in cluster_playlists[:3]:  # Mostrar primeros 3
        playlist = generator.get_playlist(name)
        
        print(f"\n{'='*60}")
        print(f"{name}")
        print(f"{'='*60}")
        
        # Características del cluster
        if 'typical_hour' in playlist.columns:
            print(f"  Hora típica: {playlist['typical_hour'].mean():.1f}h")
        if 'avg_duration' in playlist.columns:
            print(f"  Duración promedio: {playlist['avg_duration'].mean()/60000:.1f} min")
        if 'skip_rate' in playlist.columns:
            print(f"  Skip rate promedio: {playlist['skip_rate'].mean():.1%}")
        
        # Muestra de tracks
        display_cols = ['track_name', 'artist', 'play_count']
        display_cols = [c for c in display_cols if c in playlist.columns]
        print(f"\n  Top 5 tracks:")
        print(playlist[display_cols].head(5).to_string(index=False))

## 5. Exportar Resultados

### 5.1 Exportar Recomendaciones

In [ ]:
# Exportar recomendaciones híbridas
output_dir = Path('../data/features')
output_dir.mkdir(parents=True, exist_ok=True)

# Guardar recomendaciones
recs_output = output_dir / 'recommendations_hybrid.parquet'
recs_hybrid.to_parquet(recs_output, index=False)
print(f"✓ Recomendaciones guardadas en {recs_output}")

### 5.2 Exportar Playlists

In [ ]:
# Exportar playlists favoritas
playlists_dir = Path('../data/playlists')
playlists_dir.mkdir(parents=True, exist_ok=True)

playlists_to_export = ['Morning Energy', 'Never Skip Hits', 'All-Time Favorites']

for name in playlists_to_export:
    if name in generator.playlists:
        output_file = playlists_dir / f"{name.replace(' ', '_').lower()}.csv"
        generator.export_playlist(name, str(output_file))

print(f"\n✓ {len(playlists_to_export)} playlists exportadas a {playlists_dir}")

## 6. Insights y Conclusiones

### 6.1 Resumen del Sistema de Recomendación

In [ ]:
print("\n" + "="*70)
print("RESUMEN DEL SISTEMA DE RECOMENDACIÓN")
print("="*70)

print("\n📊 Estadísticas del catálogo:")
print(f"  • Total de tracks analizados: {len(recommender.track_features):,}")
print(f"  • Tracks con suficiente data (>3 plays): {len(recommender.track_features[recommender.track_features['total_plays'] >= 3]):,}")

if 'artist' in recommender.track_features.columns:
    print(f"  • Artistas únicos: {recommender.track_features['artist'].nunique():,}")

print("\n🎯 Características del usuario:")
if 'skip_tolerance' in recommender.user_profile:
    print(f"  • Skip tolerance: {recommender.user_profile['skip_tolerance']}")
if 'peak_hours' in recommender.user_profile:
    print(f"  • Horas pico de escucha: {recommender.user_profile['peak_hours']}")
if 'avg_skip_rate' in recommender.user_profile:
    print(f"  • Skip rate promedio: {recommender.user_profile['avg_skip_rate']:.1%}")

print("\n✨ Estrategias de recomendación disponibles:")
print("  • Similar: Basada en tracks similares a favoritos")
print("  • Context: Basada en hora del día y contexto temporal")
print("  • Skip-resistant: Tracks con baja probabilidad de skip")
print("  • Hybrid: Combinación de todas las estrategias")

### 6.2 Resumen de Playlists Generadas

In [ ]:
print("\n" + "="*70)
print("RESUMEN DE PLAYLISTS GENERADAS")
print("="*70)

# Agrupar por categoría
categories = {
    'Temporales': ['Morning Energy', 'Afternoon Vibes', 'Evening Chill', 'Late Night', 
                   'Weekday Focus', 'Weekend Mood'],
    'Por Comportamiento': ['Never Skip Hits', 'Deep Focus', 'Quick Hits', 
                          'All-Time Favorites', 'Shuffle Favorites'],
    'Por Mood': ['High Energy', 'Relaxation', 'Anytime Classics'],
    'Descubrimiento': ['Rediscover']
}

for category, playlist_names in categories.items():
    found = [name for name in playlist_names if name in generator.playlists]
    if found:
        print(f"\n📁 {category}: {len(found)} playlists")
        for name in found:
            count = len(generator.get_playlist(name))
            print(f"  • {name}: {count} tracks")

print(f"\n✓ Total: {len(generator.playlists)} playlists únicas generadas")

## 7. Próximos Pasos

### Mejoras Potenciales:

1. **🎨 Features Avanzados**
   - Integrar datos de audio (tempo, energy, valence)
   - Análisis de letras (sentiment analysis)
   - Features de género musical

2. **🤖 Modelos Avanzados**
   - Collaborative filtering con matriz de usuario-track
   - Deep learning (embeddings con neural networks)
   - Reinforcement learning para optimización en tiempo real

3. **📊 Evaluación**
   - A/B testing de recomendaciones
   - Métricas de engagement (tiempo de escucha, completion rate)
   - Feedback loop con ratings de usuario

4. **🚀 Producción**
   - API REST para servir recomendaciones
   - Cache de recomendaciones pre-calculadas
   - Sistema de actualización incremental
   - Monitoreo de performance y drift

5. **💡 Features de Negocio**
   - Recomendaciones por mood/actividad
   - Playlists colaborativas
   - Descubrimiento de artistas emergentes
   - Personalización por contexto (ubicación, clima, etc.)

---

**🎵 Este sistema demuestra:**
- Content-based recommendation engine completo
- Generación automática de playlists temáticas
- Clustering y análisis de patrones de escucha
- Evaluación de calidad de recomendaciones
- Production-ready code con evaluación y métricas